In [1]:
!git clone https://github.com/patil-suraj/question_generation.git

Cloning into 'question_generation'...
remote: Enumerating objects: 268, done.
remote: Total 268 (delta 0), reused 0 (delta 0), pack-reused 268
Receiving objects: 100% (268/268), 299.04 KiB | 1.99 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [31]:
%cd question_generation

/home/akihito/Desktop/mestrado/WSQASA/notebooks/create_synthetic_dataset/question_generation


In [35]:
import os 
import sys
cur_path = os.getcwd()
sys.path.append(cur_path) 
import shutil
from sklearn.model_selection import train_test_split
from pipelines import pipeline
from tqdm import tqdm
import pandas as pd
import re

/home/akihito/Desktop/mestrado/WSQASA/notebooks/create_synthetic_dataset/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [46]:
input_file_path="../../../data/tweet_qa_train_form.pkl"
output_file_path="../../../data/tweet_qa_train_QG_data.pkl"


In [37]:
df = pd.read_pickle(input_file_path)[['id', 'context']]
contexts = df['context'].unique()

In [38]:
df.head(3)

,id,context
0,0c871b7e5320d0816d5b2979d67c2649,"Our prayers are with the students, educators &..."
1,02505974856f47bbe8b487c57778de1c,I can finally say it out loud and proud: I'm g...
2,d16eb85d141d5a87bfbc438afbcf50aa,KAINE IS ABLE!!!— Cory Booker (@CoryBooker) Ju...


In [39]:
nlp = pipeline("question-generation")

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [42]:
ans_questions_map = {}
for context in tqdm(contexts):
  L_questions = []
  try:
    L_questions = nlp(context)
  except:
    pass
  ans_questions_map[context] = L_questions

  0%|          | 0/5699 [00:00<?, ?it/s]/home/akihito/Desktop/mestrado/WSQASA/notebooks/create_synthetic_dataset/.venv/lib/python3.7/site-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size
100%|██████████| 5699/5699 [2:17:49<00:00,  1.45s/it]  


In [43]:
L_id = []
L_context = []
L_question = []
L_possible_ans = []
for _, row in df.iterrows():
  for cur_pred in ans_questions_map[row['context']]:  
    L_id.append(row['id'])
    L_context.append(row['context'])
    L_question.append(cur_pred['question'])
    L_possible_ans.append(cur_pred['answer'])

In [44]:
df_question_gen = pd.DataFrame({'id' : L_id,
                                'context' : L_context,
                                'question' : L_question,
                                'answers': L_possible_ans})

In [45]:
df_question_gen.head(3)


,id,context,question,answers
0,0c871b7e5320d0816d5b2979d67c2649,"Our prayers are with the students, educators &...","Where are our prayers with students, educators...",Independence High School
1,0c871b7e5320d0816d5b2979d67c2649,"Our prayers are with the students, educators &...",Who is the name of the #PatriotPride?,Doug Ducey
2,0c871b7e5320d0816d5b2979d67c2649,"Our prayers are with the students, educators &...",Who is the name of the #PatriotPride?,Doug Ducey


In [47]:
df_question_gen.to_pickle(output_file_path)